In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df= pd.read_csv('vgames2.csv')

**EDA**

In [ ]:
#Unnamed: 0 컬럼 삭제
df.drop('Unnamed: 0'	,axis=1,inplace=True)

In [ ]:
#결측치 없는지 확인
missing_values_count = df.isnull().sum()
missing_values_count 
# Year/Genre/Publisher 결측치 존재함

매출 전처리

In [ ]:
df[df["NA_Sales"].str.endswith("M")]
df["NA_Sales"]=df["NA_Sales"].str.replace('M', '')
a=df[df["NA_Sales"].str.endswith("K")]
a["NA_Sales"]=a["NA_Sales"].str.replace('K', '')
a["NA_Sales"].value_counts()
a["NA_Sales"] = pd.to_numeric(a["NA_Sales"])
a["NA_Sales"]=a["NA_Sales"]*0.001
df.update(a,overwrite=True)
df["NA_Sales"]=pd.to_numeric(df["NA_Sales"])

In [ ]:
df[df["EU_Sales"].str.endswith("M")]
df["EU_Sales"]=df["EU_Sales"].str.replace('M', '')
a=df[df["EU_Sales"].str.endswith("K")]
a["EU_Sales"]=a["EU_Sales"].str.replace('K', '')
a["EU_Sales"].value_counts()
a["EU_Sales"] = pd.to_numeric(a["EU_Sales"])
a["EU_Sales"]=a["EU_Sales"]*0.001
df.update(a,overwrite=True)
df["EU_Sales"] = pd.to_numeric(df["EU_Sales"])

In [ ]:
df[df["JP_Sales"].str.endswith("M")]
df["JP_Sales"]=df["JP_Sales"].str.replace('M', '')
a=df[df["JP_Sales"].str.endswith("K")]
a["JP_Sales"]=a["JP_Sales"].str.replace('K', '')
a["JP_Sales"].value_counts()
a["JP_Sales"] = pd.to_numeric(a["JP_Sales"])
a["JP_Sales"]=a["JP_Sales"]*0.001
df.update(a,overwrite=True)
df["JP_Sales"] = pd.to_numeric(df["JP_Sales"])

In [ ]:
df[df["Other_Sales"].str.endswith("M")]
df["Other_Sales"]=df["Other_Sales"].str.replace('M', '')
a=df[df["Other_Sales"].str.endswith("K")]
a["Other_Sales"]=a["Other_Sales"].str.replace('K', '')
a["Other_Sales"].value_counts()
a["Other_Sales"] = pd.to_numeric(a["Other_Sales"])
a["Other_Sales"]=a["Other_Sales"]*0.001
df.update(a,overwrite=True)
df["Other_Sales"] = pd.to_numeric(df["Other_Sales"])

In [ ]:
df['sales_total'] = (df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] ) 

In [ ]:
#1. year 6자리인지 확인 후 아닌것 수기로 작성
#2. year 6자리인것들 .0제거
#3. year datatime으로 변경 
#4. year 일정주기 기준으로 나누기
#16597 총 데이터 갯수

In [ ]:
df.Year=df.Year.astype(str)

In [ ]:
df_6=df[df.Year.str.len()==6] #-> 0000형식으로 변환
df_4=df[df.Year.str.len()==4] #47 -> 0000형식으로 따로 변환
df_3=df[df.Year.str.len()==3] #323 nan값 -> nan값 name platform 중복되는 값은 서치를 통해 수기로 작성 나머지는 삭제
#len에 따라 if else문 작성하는 법 시간남으면 해보기 

In [ ]:
df_6.Year= df_6.Year.astype(str).str[:-2]

In [ ]:
df_4.Year=df_4.Year.astype(str).str[:-2]

In [ ]:
df_4.Year = pd.to_numeric(df_4.Year)

In [ ]:
df_4.Year_19=1900+df_4.Year[df_4.Year>23]

In [ ]:
df_4.Year_20=2000+df_4.Year[df_4.Year<=23]

In [ ]:
df_4.Year = pd.concat([df_4.Year_19,df_4.Year_20])
df_4

In [ ]:
df_3.Year=np.nan
#버리기보다는 NAN값으로 둠으로 데이터 손실을 방지한다

In [ ]:
df_non_nan = pd.concat([df_4,df_6])

In [ ]:
#=df_year_non_nan.Year.astype(np.datetime64)
df_non_nan.Year = pd.to_numeric(df_non_nan.Year)

In [ ]:
#중복값 처리
#SALES 합치기
df_non_nan[df_non_nan.duplicated(subset=['Name','Platform'],keep=False )]
#같은 이름의 게임이 존재해도 모두 같은 연도나 같은 플랫폼이 아니기 때문에 중복값 처리하지 않는다 

In [ ]:
#결측치 중 Publisher같은 경우에는 게임 제목을 통해 알 수 있는 것은 처리해준다

In [ ]:
a= df_non_nan.groupby('Year')['sales_total'].sum()
a= a.reset_index()

In [ ]:
b= df_non_nan.groupby('Year')['NA_Sales'].sum()
b= b.reset_index()

In [ ]:
c= df_non_nan.groupby('Year')['EU_Sales'].sum()
c= c.reset_index()

In [ ]:
d= df_non_nan.groupby('Year')['JP_Sales'].sum()
d= d.reset_index()

In [ ]:
a_b =pd.merge(a,b, how='outer',on='Year')

In [ ]:
a_b_c= pd.merge(a_b,c, how='outer',on='Year')

In [ ]:
df_year= pd.merge(a_b_c,d, how='outer',on='Year')

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df_year['Year'], df_year['sales_total'], label='sales total')
plt.plot(df_year['Year'], df_year['NA_Sales'], label='NA_Sales')
plt.plot(df_year['Year'], df_year['EU_Sales'], label='EU_Sales')
plt.plot(df_year['Year'], df_year['JP_Sales'], label='JP_Sales')
plt.title('Year of Sale')
plt.xlabel('Year')
plt.ylabel('sales_total')
plt.legend()
plt.show()

In [ ]:
#연도를 기준을 통해 지정해주었다
year_80=df_non_nan.query("Year >=1980").query( "Year <1990" )
year_90=df_non_nan.query("Year >=1990").query( "Year <2000" )
year_00=df_non_nan.query("Year >=2000").query( "Year <2005" )
year_05=df_non_nan.query("Year >=2005").query( "Year <2010" )
year_10=df_non_nan.query("Year >=2010").query( "Year <2015" )
year_15=df_non_nan.query("Year >=2015")

In [ ]:
year_80.Year='1980'
year_90.Year= '1990'
year_00.Year= '2000'
year_05.Year= '2005'
year_10.Year= '2010'
year_15.Year= '2015'

In [ ]:
df_clean=pd.concat([year_80,year_90,year_00,year_05,year_10,year_15])

In [ ]:
df_1980=df_clean[df_clean.Year=='1980']
Genre=df_1980.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=df_1980.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(3)
publisher=df_1980.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
#name=df_1980.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)

In [ ]:
df_top3_Publisher = df_1980.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(3)
df_top3_Publisher= df_top3_Publisher.reset_index()
plt.figure(figsize=(10,5))
sns.barplot(x="Publisher", y="sales_total", data=df_top3_Publisher)
plt.title('Top 3 Publisher by1980')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
ratio = [66, 32, 30]
labels = ['Action ', 'Platform', 'Shooter ']
explode = [0.05, 0.05, 0.05]
colors = ['lightskyblue', 'royalblue', 'lightgrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 3 genre 1980')
plt.show()
ratio = [116, 74, 11]
labels = ['2600 ', 'NES', 'GB ']
explode = [0.05, 0.05, 0.05]
colors = ['#ff9999', '#ffc000', '#d395d0']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 3 Platform 1980')
plt.show()

In [ ]:
df_1990=df_clean[df_clean.Year=='1990']
Genre=df_1990.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=df_1990.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(4)
publisher=df_1990.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
publisher= publisher.reset_index()
#name=df_1990.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
plt.figure(figsize=(13,5))
sns.barplot(x="Publisher", y="sales_total", data=publisher)
plt.title('Top 5 Publisher by1990')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
Genre= Genre.reset_index()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(Genre.sales_total, labels=Genre.Genre, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 genre 1990')
plt.show()
platform=platform.reset_index()
platform=platform.reset_index()
colors = ['#ff9999', '#ffc000', '#d395d0','#8fd9b6']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(platform.sales_total, labels=platform.Platform, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 4 platform 1990')
plt.show()

In [ ]:
df_2000=df_clean[df_clean.Year=='2000']
Genre=df_2000.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
Genre= Genre.reset_index()
platform=df_2000.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=platform.reset_index()
publisher=df_2000.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
publisher= publisher.reset_index()
name=df_2000.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
plt.figure(figsize=(10,5))
sns.barplot(x="Publisher", y="sales_total", data=publisher)
plt.title('Top 3 Publisher by 2000')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(Genre.sales_total, labels=Genre.Genre, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 genre 2000')
plt.show()
platform=platform.reset_index()
colors = ['#ff9999', '#ffc000', '#d395d0','#8fd9b6','darkorange']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(platform.sales_total, labels=platform.Platform, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 platform 2000')
plt.show()

In [ ]:
df_2005=df_clean[df_clean.Year=='2005']
Genre=df_2005.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
Genre= Genre.reset_index()
platform=df_2005.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=platform.reset_index()
publisher=df_2005.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
publisher= publisher.reset_index()
name=df_2005.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
plt.figure(figsize=(10,5))
sns.barplot(x="Publisher", y="sales_total", data=publisher)
plt.title('Top 5 Publisher by 2005')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(Genre.sales_total, labels=Genre.Genre, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 genre 2005')
plt.show()
platform=platform.reset_index()
colors = ['#ff9999', '#ffc000', '#d395d0','#8fd9b6','darkorange']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(platform.sales_total, labels=platform.Platform, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 platform 2005')
plt.show()

In [ ]:
df_2010=df_clean[df_clean.Year=='2010']
Genre=df_2010.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
Genre= Genre.reset_index()
platform=df_2010.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=platform.reset_index()
publisher=df_2010.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
publisher= publisher.reset_index()
name=df_2010.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
plt.figure(figsize=(10,5))
sns.barplot(x="Publisher", y="sales_total", data=publisher)
plt.title('Top 5 Publisher by 2010')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(Genre.sales_total, labels=Genre.Genre, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 genre 2010')
plt.show()
platform=platform.reset_index()
colors = ['#ff9999', '#ffc000', '#d395d0','#8fd9b6','darkorange']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(platform.sales_total, labels=platform.Platform, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 platform 2010')
plt.show()

In [ ]:
df_2015=df_clean[df_clean.Year=='2015']
Genre=df_2015.groupby(['Year', 'Genre'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
Genre= Genre.reset_index()
platform=df_2015.groupby(['Year', 'Platform'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
platform=platform.reset_index()
publisher=df_2015.groupby(['Year', 'Publisher'])['sales_total'].sum().groupby(level=0, group_keys=False).nlargest(5)
publisher= publisher.reset_index()
name=df_2015.groupby(['Year', 'Name'])['sales_total'].count().groupby(level=0, group_keys=False).nlargest(5)
plt.figure(figsize=(14,5))
sns.barplot(x="Publisher", y="sales_total", data=publisher)
plt.title('Top 5 Publisher by 2015')
plt.xlabel('Publisher')
plt.ylabel('sales_total')
plt.show()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(Genre.sales_total, labels=Genre.Genre, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 genre 2015')
plt.show()
platform=platform.reset_index()
colors = ['#ff9999', '#ffc000', '#d395d0','#8fd9b6','darkorange']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(platform.sales_total, labels=platform.Platform, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 platform 2015')
plt.show()

연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
스포츠가 인기가 1995년부터 올라가더니 점점 인기가 줄고 있다
또한 액션 역신 2000년대부터 선풍적으로 인기를 끌더니 감소하고 있다
하지만 이는 게임 자체의 인가가 줄은것인지 이 장르들이 줄은 것인지 확신할 수 없기 때문에 normalization을 통해 그래프를 다시 그려 확인해보아야한다

트렌드는 어떤 것을 기준으로 파악했나요?

연도별로 트렌드가 존재하나요?

왜 존재한다/존재하지 않는다고 생각했나요?

In [ ]:
df_na=df_clean[df_clean.NA_Sales>0]
#df_na.iloc[]
A=df_na.groupby('Genre').count()
sr=A.NA_Sales.nlargest(5)
sr
sr.plot(kind='bar',figsize=(8,6),title='NA Genre top 5',legend=True,grid=True,ylim=(0,3000),rot=45)
plt.show()
#na는 Action>Sports>Misc>Shooter>Racing

In [ ]:
df_Action=df_clean[df_clean.Genre	=='Sports']
df_Action= df_non_nan.groupby('Name')['sales_total'].sum()
df_Action= df_Action.reset_index()
df_Action

In [ ]:
df_Action=df_clean[df_clean.Genre	=='Action']
df_Action= df_non_nan.groupby('Name')['sales_total'].sum()
df_Action= df_Action.reset_index()
df_Action

In [ ]:
df_EU=df_clean[df_clean.EU_Sales>0]
#df_na.iloc[]
A=df_EU.groupby('Genre').count()
sr=A.EU_Sales.nlargest(5)
sr
sr.plot(kind='bar',figsize=(8,6),title='EU Genre top 5',legend=True,grid=True,ylim=(0,3000),rot=45,color ='C1')
plt.show()

In [ ]:
df_JP=df_clean[df_clean.JP_Sales>0]
#df_na.iloc[]
A=df_JP.groupby('Genre').count()
sr=A.JP_Sales.nlargest(5)
sr
sr.plot(kind='bar',figsize=(8,6),title='JP Genre top 5',legend=True,grid=True,ylim=(0,3000),rot=45,color='limegreen')
plt.show()
#na는 Action>Role-Playing>Adventure>Sports>Misc

In [ ]:
df_Other=df_clean[df_clean.Other_Sales>0]
#df_na.iloc[]
A=df_Other.groupby('Genre').count()
sr=A.Other_Sales.nlargest(5)
sr
sr.plot(kind='bar',figsize=(8,6),title='Other Genre top 5',legend=True,grid=True,ylim=(0,3000),rot=45,color='lightpink')
plt.show()
#na는 Action>Role-Playing>Adventure>Sports>Misc

In [ ]:
df_Name_top3 = df.groupby(['Name','Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales'])['sales_total'].sum().nlargest(10)
df_Name_top3 = df_Name_top3.reset_index()
df_Name_top3

In [ ]:
df_Name_top3 = df.groupby(['Name','Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales'])['sales_total'].sum().nsmallest(10)
df_Name_top3 = df_Name_top3.reset_index()
df_Name_top3

In [ ]:
#년도별 트랜드/ 어느 나라가 sales가장 큰지, 년도별로 어떤 플랫폼 퍼블리셔가 가장 많았는지 확ㅇ니 
df_Name_top3 = df_non_nan.groupby(['Name','Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales'])['sales_total'].sum().nlargest(5)
df_Name_top3 = df_Name_top3.reset_index()
plt.figure(figsize=(20,10))
sns.barplot(x="Name", y="sales_total", data=df_Name_top3)
plt.title('Top 5 game')
plt.ylabel('sales_total')
plt.show()

In [ ]:
df_Name_top30= df_non_nan.groupby(['Name','Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales'])['sales_total'].sum().nlargest(10)
df_Name_top30 = df_Name_top30.reset_index()
df_Name_top30.Genre.value_counts()

In [ ]:
df_Name_top30

In [ ]:
df_SUM= df_non_nan.groupby('Genre')['sales_total'].sum()
df_SUM= df_SUM.reset_index()
df_SUM
#난텐도 회사 압도적

In [ ]:
df_COUNT= df_non_nan.groupby('Genre')['sales_total'].count()
df_COUNT= df_COUNT.reset_index()
df_COUNT.rename(columns = {"sales_total": "count"}, inplace = True)
df_COUNT

In [ ]:
df_genre = pd.merge(df_SUM,df_COUNT, how='outer',on='Genre')
df_genre['장르당 매출']=round(df_genre['sales_total']/df_genre['count'],2)
ab=df_genre.sort_values('장르당 매출', ascending=False)
ab

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x="Genre", y="장르당 매출", data=ab)
plt.title('best sales genre')
plt.ylabel('genre per sales')
plt.show()

In [ ]:
df_Publisher_top5= df_non_nan.groupby('Publisher')['sales_total'].sum().nlargest(5)
df_Publisher_top5=df_Publisher_top5.reset_index()
colors = ['lightskyblue', 'royalblue', 'blue','lightgrey','slategrey']
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(df_Publisher_top5.sales_total, labels=df_Publisher_top5.Publisher, autopct='%.1f%%', startangle=260, counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.title('Top 5 Publisher')
plt.show()

In [ ]:
df_Nintendo =df_non_nan[df_non_nan.Publisher=='Nintendo']
abc= df_Nintendo.groupby('Genre')['sales_total'].count().nlargest(5)
abc=abc.reset_index()
abc.rename(columns = {"sales_total": "count"}, inplace = True)
abc

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x="Genre", y="count", data=abc)
plt.title('Top 5 Genre Nintendo')
plt.ylabel('genre of Nintendo')
plt.show()

In [ ]:
df_Electronic =df_non_nan[df_non_nan.Publisher=='Electronic Arts']
abc= df_Electronic.groupby('Genre')['sales_total'].count().nlargest(5)
abc=abc.reset_index()
abc.rename(columns = {"sales_total": "count"}, inplace = True)
abc
plt.figure(figsize=(10,5))
sns.barplot(x="Genre", y="count", data=abc)
plt.title('Top 5 Genre Electronic Arts')
plt.ylabel('genre of Electronic Arts')
plt.show()

In [ ]:
df_Electronic.shape,df_Nintendo.shape

In [ ]:
df_Activision =df_non_nan[df_non_nan.Publisher=='Activision']
abc= df_Nintendo.groupby('Genre')['sales_total'].count().nlargest(5)
abc=abc.reset_index()
abc.rename(columns = {"sales_total": "count"}, inplace = True)
plt.figure(figsize=(10,5))
sns.barplot(x="Genre", y="count", data=abc)
plt.title('Top 5 Genre Activision')
plt.ylabel('genre of Activision')
plt.show()